# Working my Way Through Some of the NBA API Documentation

https://github.com/swar/nba_api/tree/master/docs/examples 

## Basics and Package Structure

In [6]:
from nba_api.stats.endpoints import playercareerstats
# Anthony Davis
career = playercareerstats.PlayerCareerStats(player_id='203076')
career.get_data_frames()[0]

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,203076,2012-13,00,1610612740,NOH,20.0,64,60,1846.0,349,676,0.516,0,6,0.000,169,225,0.751,165,357,522,63,75,112,89,158,867
1,203076,2013-14,00,1610612740,NOP,21.0,67,66,2358.0,522,1005,0.519,2,9,0.222,348,440,0.791,207,466,673,105,89,189,109,200,1394
2,203076,2014-15,00,1610612740,NOP,22.0,68,68,2455.0,642,1199,0.535,1,12,0.083,371,461,0.805,173,523,696,149,100,200,95,141,1656
3,203076,2015-16,00,1610612740,NOP,23.0,61,61,2164.0,560,1136,0.493,35,108,0.324,326,430,0.758,130,497,627,116,78,125,121,148,1481
4,203076,2016-17,00,1610612740,NOP,24.0,75,75,2708.0,770,1526,0.505,40,134,0.299,519,647,0.802,172,712,884,157,94,167,181,168,2099
5,203076,2017-18,00,1610612740,NOP,25.0,75,75,2727.0,780,1462,0.534,55,162,0.340,495,598,0.828,187,644,831,174,115,193,162,159,2110
6,203076,2018-19,00,1610612740,NOP,26.0,56,56,1850.0,530,1026,0.517,48,145,0.331,344,433,0.794,174,498,672,218,88,135,112,132,1452
7,203076,2019-20,00,1610612747,LAL,27.0,62,62,2131.0,551,1096,0.503,72,218,0.330,444,525,0.846,142,435,577,200,91,143,154,156,1618
8,203076,2020-21,00,1610612747,LAL,28.0,36,36,1162.0,301,613,0.491,26,100,0.260,158,214,0.738,62,224,286,110,45,59,74,60,786
9,203076,2021-22,00,1610612747,LAL,29.0,40,40,1404.0,370,695,0.532,13,70,0.186,174,244,0.713,106,288,394,122,49,90,82,97,927


## Getting Team and Player IDs

In [11]:
from nba_api.stats.static import teams
# get_teams returns a list of 30 dictionaries, each an NBA team
nba_teams = teams.get_teams()
print(f"Number of Teams Fetched: {len(nba_teams)}")
nba_teams[:3]

Number of Teams Fetched: 30


[{'id': 1610612737,
  'full_name': 'Atlanta Hawks',
  'abbreviation': 'ATL',
  'nickname': 'Hawks',
  'city': 'Atlanta',
  'state': 'Atlanta',
  'year_founded': 1949},
 {'id': 1610612738,
  'full_name': 'Boston Celtics',
  'abbreviation': 'BOS',
  'nickname': 'Celtics',
  'city': 'Boston',
  'state': 'Massachusetts',
  'year_founded': 1946},
 {'id': 1610612739,
  'full_name': 'Cleveland Cavaliers',
  'abbreviation': 'CLE',
  'nickname': 'Cavaliers',
  'city': 'Cleveland',
  'state': 'Ohio',
  'year_founded': 1970}]

In [12]:
from nba_api.stats.static import players
# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
print(f"Number of Players Fetched: {len(nba_players)}")
nba_players[:3]

Number of Players Fetched: 4831


[{'id': 76001,
  'full_name': 'Alaa Abdelnaby',
  'first_name': 'Alaa',
  'last_name': 'Abdelnaby',
  'is_active': False},
 {'id': 76002,
  'full_name': 'Zaid Abdul-Aziz',
  'first_name': 'Zaid',
  'last_name': 'Abdul-Aziz',
  'is_active': False},
 {'id': 76003,
  'full_name': 'Kareem Abdul-Jabbar',
  'first_name': 'Kareem',
  'last_name': 'Abdul-Jabbar',
  'is_active': False}]

In [17]:
# Use dictionary to search for an individual or player by name (or any other attribute)
spurs = [team for team in nba_teams if team['full_name'] == 'San Antonio Spurs'][0]

# spurs is now the dictionary with information on the San Antonio Spurs
print(spurs)

{'id': 1610612759, 'full_name': 'San Antonio Spurs', 'abbreviation': 'SAS', 'nickname': 'Spurs', 'city': 'San Antonio', 'state': 'Texas', 'year_founded': 1976}


In [18]:
big_fundamental = [player for player in nba_players if player['full_name'] == 'Tim Duncan'][0]

# big_fundamental is now the dictionary with the information on Tim Duncan
print(big_fundamental)

{'id': 1495, 'full_name': 'Tim Duncan', 'first_name': 'Tim', 'last_name': 'Duncan', 'is_active': False}


## Finding Games

Let's find the Celtics game vs the Raptors in 2017-18
1. Fetch all Celtics games
2. Select just games from the 2017-2018 season (SEASON_ID ending in 2017)
3. Select games where the opponent is the Raptors (MATCHUP contains 'TOR')
4. Order by date and select the last row

We can also get play-by-play data from that game

In [20]:
celtics = [team for team in nba_teams if team['full_name'] == 'Boston Celtics'][0]
celtics_id = celtics['id']

In [23]:
from nba_api.stats.endpoints import leaguegamefinder

# Query for games where the Celtics were playing
gamefinder = leaguegamefinder.LeagueGameFinder(team_id_nullable = celtics_id)
# The first df of those returned is what we want
games = gamefinder.get_data_frames()[0]
games.head()


,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
0,22022,1610612738,BOS,Boston Celtics,0022200921,2023-02-27,BOS @ NYK,L,242,94,37,84,0.440,9,42,0.214,11,14,0.786,7,28,35,21,5,8,11,22,-15.0
1,22022,1610612738,BOS,Boston Celtics,0022200908,2023-02-25,BOS @ PHI,W,239,110,43,86,0.500,16,36,0.444,8,12,0.667,9,35,44,20,6,4,12,22,3.0
2,22022,1610612738,BOS,Boston Celtics,0022200887,2023-02-23,BOS @ IND,W,266,142,51,99,0.515,18,39,0.462,22,26,0.846,20,31,51,31,7,6,15,18,4.0
3,22022,1610612738,BOS,Boston Celtics,0022200877,2023-02-15,BOS vs. DET,W,240,127,49,92,0.533,21,45,0.467,8,10,0.800,8,35,43,30,10,5,11,18,18.0
4,22022,1610612738,BOS,Boston Celtics,0022200869,2023-02-14,BOS @ MIL,L,266,125,46,99,0.465,17,50,0.340,16,20,0.800,8,36,44,28,8,1,13,26,-6.0


In [24]:
# Subset the games to when the last 4 digits of the SEASON_ID were 2017
games_1718 = games[games.SEASON_ID.str[-4:] == '2017']
games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
469,42017,1610612738,BOS,Boston Celtics,0041700307,2018-05-27,BOS vs. CLE,L,240,79,29,85,0.341,7,39,0.179,14,19,0.737,11,31,42,18,6,0,5,20,-8.0
470,42017,1610612738,BOS,Boston Celtics,0041700306,2018-05-25,BOS @ CLE,L,241,99,38,74,0.514,12,28,0.429,11,20,0.550,5,26,31,25,5,2,13,18,-10.0
471,42017,1610612738,BOS,Boston Celtics,0041700305,2018-05-23,BOS vs. CLE,W,240,96,31,85,0.365,13,39,0.333,21,23,0.913,7,38,45,18,10,6,8,19,13.0
472,42017,1610612738,BOS,Boston Celtics,0041700304,2018-05-21,BOS @ CLE,L,239,102,35,85,0.412,9,28,0.321,23,30,0.767,9,28,37,21,9,3,9,26,-9.0
473,42017,1610612738,BOS,Boston Celtics,0041700303,2018-05-19,BOS @ CLE,L,240,86,29,74,0.392,6,22,0.273,22,28,0.786,6,28,34,16,4,4,15,25,-30.0


In [25]:
# Now filter to just games against the Raptors
raps_games_1718 = games_1718[games_1718.MATCHUP.str.contains('TOR')]
raps_games_1718.head()

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
492,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0
494,22017,1610612738,BOS,Boston Celtics,0021701140,2018-03-31,BOS vs. TOR,W,240,110,40,88,0.455,7,17,0.412,23,27,0.852,8,33,41,19,6,3,5,23,11.0
515,22017,1610612738,BOS,Boston Celtics,0021700798,2018-02-06,BOS @ TOR,L,240,91,33,83,0.398,10,23,0.435,15,20,0.750,9,29,38,21,3,3,17,19,-20.0
556,22017,1610612738,BOS,Boston Celtics,0021700188,2017-11-12,BOS vs. TOR,W,239,95,35,87,0.402,10,26,0.385,15,19,0.789,15,31,46,24,9,2,14,18,1.0


In [26]:
last_raps_game = raps_games_1718.sort_values('GAME_DATE').iloc[-1]
last_raps_game

SEASON_ID                     22017
TEAM_ID                  1610612738
TEAM_ABBREVIATION               BOS
TEAM_NAME            Boston Celtics
GAME_ID                  0021701171
GAME_DATE                2018-04-04
MATCHUP                   BOS @ TOR
WL                                L
MIN                             237
PTS                              78
FGM                              25
FGA                              75
FG_PCT                        0.333
FG3M                              3
FG3A                             22
FG3_PCT                       0.136
FTM                              25
FTA                              28
FT_PCT                        0.893
OREB                              8
DREB                             35
REB                              43
AST                              10
STL                               7
BLK                               3
TOV                              16
PF                               16
PLUS_MINUS                  

In [27]:
game_id = last_raps_game.GAME_ID
game_id

'0021701171'

In [28]:
# Get **all** the games so we can filter to an individual GAME_ID
result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]
# Find all GAME_ID we want
full_game = all_games[all_games.GAME_ID == game_id]
full_game

,SEASON_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_NAME,GAME_ID,GAME_DATE,MATCHUP,WL,MIN,PTS,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PLUS_MINUS
26359,22017,1610612738,BOS,Boston Celtics,0021701171,2018-04-04,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0
26363,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,40,92,0.435,10,33,0.303,6,8,0.750,12,36,48,23,10,6,10,25,18.0


In [29]:
# It is very common to want to join these two dataframes to get stats for both teams as opposed to just one team in a single df and the other team in a different df
# Luckily the GitHub documentation contains a function to do this...

import pandas as pd

def combine_team_games(df, keep_method='home'):
    '''Combine a TEAM_ID-GAME_ID unique table into rows by game. Slow.

        Parameters
        ----------
        df : Input DataFrame.
        keep_method : {'home', 'away', 'winner', 'loser', ``None``}, default 'home'
            - 'home' : Keep rows where TEAM_A is the home team.
            - 'away' : Keep rows where TEAM_A is the away team.
            - 'winner' : Keep rows where TEAM_A is the losing team.
            - 'loser' : Keep rows where TEAM_A is the winning team.
            - ``None`` : Keep all rows. Will result in an output DataFrame the same
                length as the input DataFrame.
                
        Returns
        -------
        result : DataFrame
    '''
    # Join every row to all others with the same game ID.
    joined = pd.merge(df, df, suffixes=['_A', '_B'],
                      on=['SEASON_ID', 'GAME_ID', 'GAME_DATE'])
    # Filter out any row that is joined to itself.
    result = joined[joined.TEAM_ID_A != joined.TEAM_ID_B]
    # Take action based on the keep_method flag.
    if keep_method is None:
        # Return all the rows.
        pass
    elif keep_method.lower() == 'home':
        # Keep rows where TEAM_A is the home team.
        result = result[result.MATCHUP_A.str.contains(' vs. ')]
    elif keep_method.lower() == 'away':
        # Keep rows where TEAM_A is the away team.
        result = result[result.MATCHUP_A.str.contains(' @ ')]
    elif keep_method.lower() == 'winner':
        result = result[result.WL_A == 'W']
    elif keep_method.lower() == 'loser':
        result = result[result.WL_A == 'L']
    else:
        raise ValueError(f'Invalid keep_method: {keep_method}')
    return result
    
# Combine the game rows into one. By default, the home team will be TEAM_A.
game_df = combine_team_games(full_game)
game_df




,SEASON_ID,TEAM_ID_A,TEAM_ABBREVIATION_A,TEAM_NAME_A,GAME_ID,GAME_DATE,MATCHUP_A,WL_A,MIN_A,PTS_A,FGM_A,FGA_A,FG_PCT_A,FG3M_A,FG3A_A,FG3_PCT_A,FTM_A,FTA_A,FT_PCT_A,OREB_A,DREB_A,REB_A,AST_A,STL_A,BLK_A,TOV_A,PF_A,PLUS_MINUS_A,TEAM_ID_B,TEAM_ABBREVIATION_B,TEAM_NAME_B,MATCHUP_B,WL_B,MIN_B,PTS_B,FGM_B,FGA_B,FG_PCT_B,FG3M_B,FG3A_B,FG3_PCT_B,FTM_B,FTA_B,FT_PCT_B,OREB_B,DREB_B,REB_B,AST_B,STL_B,BLK_B,TOV_B,PF_B,PLUS_MINUS_B
2,22017,1610612761,TOR,Toronto Raptors,0021701171,2018-04-04,TOR vs. BOS,W,240,96,40,92,0.435,10,33,0.303,6,8,0.75,12,36,48,23,10,6,10,25,18.0,1610612738,BOS,Boston Celtics,BOS @ TOR,L,237,78,25,75,0.333,3,22,0.136,25,28,0.893,8,35,43,10,7,3,16,16,-18.0


In [30]:
# With this GAME_ID, we can also fetch play-by-play data

from nba_api.stats.endpoints import playbyplayv2
pbp = playbyplayv2.PlayByPlayV2(game_id = game_id)
pbp = pbp.get_data_frames()[0]
pbp.head()

,GAME_ID,EVENTNUM,EVENTMSGTYPE,EVENTMSGACTIONTYPE,PERIOD,WCTIMESTRING,PCTIMESTRING,HOMEDESCRIPTION,NEUTRALDESCRIPTION,VISITORDESCRIPTION,SCORE,SCOREMARGIN,PERSON1TYPE,PLAYER1_ID,PLAYER1_NAME,PLAYER1_TEAM_ID,PLAYER1_TEAM_CITY,PLAYER1_TEAM_NICKNAME,PLAYER1_TEAM_ABBREVIATION,PERSON2TYPE,PLAYER2_ID,PLAYER2_NAME,PLAYER2_TEAM_ID,PLAYER2_TEAM_CITY,PLAYER2_TEAM_NICKNAME,PLAYER2_TEAM_ABBREVIATION,PERSON3TYPE,PLAYER3_ID,PLAYER3_NAME,PLAYER3_TEAM_ID,PLAYER3_TEAM_CITY,PLAYER3_TEAM_NICKNAME,PLAYER3_TEAM_ABBREVIATION,VIDEO_AVAILABLE_FLAG
0,0021701171,2,12,0,1,8:11 PM,12:00,None,Start of 1st Period (8:11 PM EST),None,None,None,0.0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,0
1,0021701171,4,10,0,1,8:11 PM,12:00,Jump Ball Valanciunas vs. Baynes: Tip to Anunoby,None,None,None,None,4.0,202685,Jonas Valanciunas,1.610613e+09,Toronto,Raptors,TOR,5,203382,Aron Baynes,1.610613e+09,Boston,Celtics,BOS,4,1628384,O.G. Anunoby,1.610613e+09,Toronto,Raptors,TOR,1
2,0021701171,7,2,1,1,8:11 PM,11:38,MISS DeRozan 21' Jump Shot,None,None,None,None,4.0,201942,DeMar DeRozan,1.610613e+09,Toronto,Raptors,TOR,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
3,0021701171,8,4,0,1,8:11 PM,11:36,None,None,Tatum REBOUND (Off:0 Def:1),None,None,5.0,1628369,Jayson Tatum,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1
4,0021701171,9,2,86,1,8:12 PM,11:21,None,None,MISS Horford 12' Turnaround Fadeaway Shot,None,None,5.0,201143,Al Horford,1.610613e+09,Boston,Celtics,BOS,0,0,None,NaN,None,None,None,0,0,None,NaN,None,None,None,1


## Working with NBA live data...

In [35]:
# Query nba.live.endpoints.scoreboard and  list games in localTimeZone
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard

f = "{gameId}: {awayTeam} vs. {homeTeam} @ {gameTimeLTZ}" 

board = scoreboard.ScoreBoard()
print("ScoreBoardDate: " + board.score_board_date)
games = board.games.get_dict()
for game in games:
    gameTimeLTZ = parser.parse(game["gameTimeUTC"]).replace(tzinfo=timezone.utc).astimezone(tz=None)
    print(f.format(gameId=game['gameId'], awayTeam=game['awayTeam']['teamName'], homeTeam=game['homeTeam']['teamName'], gameTimeLTZ=gameTimeLTZ))

ScoreBoardDate: 2023-02-28
0022200923: Wizards vs. Hawks @ 2023-02-28 19:30:00-05:00
0022200924: Bucks vs. Nets @ 2023-02-28 19:30:00-05:00
0022200925: Bulls vs. Raptors @ 2023-02-28 19:30:00-05:00
0022200927: Lakers vs. Grizzlies @ 2023-02-28 19:30:00-05:00
0022200926: Nuggets vs. Rockets @ 2023-02-28 20:00:00-05:00
0022200928: Kings vs. Thunder @ 2023-02-28 20:00:00-05:00
0022200929: Pacers vs. Mavericks @ 2023-02-28 20:30:00-05:00
0022200930: Spurs vs. Jazz @ 2023-02-28 21:00:00-05:00
0022200931: Trail Blazers vs. Warriors @ 2023-02-28 22:00:00-05:00
0022200932: Timberwolves vs. Clippers @ 2023-02-28 22:00:00-05:00


In [36]:
# Get BoxScore
from nba_api.live.nba.endpoints import boxscore
box = boxscore.BoxScore('0022200931')

In [44]:
# Data Sets
# box.game.get_dict()                   #equal to box.get_dict()['game']
# box.arena.get_dict()                  #equal to box.get_dict()['game']['arena']
# box.away_team.get_dict()              #equal to box.get_dict()['game']['awayTeam']
# box.away_team_player_stats.get_dict() #equal to box.get_dict()['game']['awayTeam']['players']
# box.away_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
# box.home_team.get_dict()              #equal to box.get_dict()['game']['homeTeam']
# box.home_team_player_stats.get_dict() #equal to box.get_dict()['game']['homeTeam']['players']
# box.home_team_stats.get_dict()        #equal to box.get_dict()['game']['homeTeam'] w/o ['players']
# box.game_details.get_dict()           #equal to box.get_dict()['game'] scrubbed of all other dictionaries
# box.officials.get_dict()              #equal to box.get_dict()['game']['officials']

In [50]:
# Getting Box Scores. 
# Note: home_team & away_team have the identical structure just swapping out variable names

away_players = box.away_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in away_players:
    print(f.format(player_id = player['personId'], name = player['name'], points = player['statistics']['points']))
print("")
home_players = box.home_team.get_dict()['players']
f = "{player_id}: {name}: {points} PTS"
for player in home_players:
    print(f.format(player_id = player['personId'], name = player['name'], points = player['statistics']['points']))


1629629: Cam Reddish: 10 PTS
203924: Jerami Grant: 24 PTS
1629234: Drew Eubanks: 5 PTS
1629680: Matisse Thybulle: 15 PTS
203081: Damian Lillard: 25 PTS
1630570: Trendon Watford: 2 PTS
1629642: Nassir Little: 7 PTS
1631101: Shaedon Sharpe: 13 PTS
1630553: Keon Johnson: 4 PTS
1628995: Kevin Knox II: 0 PTS
1631133: Jabari Walker: 0 PTS
1627853: Ryan Arcidiacono: 0 PTS
1630641: Ibou Badji: 0 PTS
1631219: John Butler Jr.: 0 PTS
203994: Jusuf Nurkic: 0 PTS
1629014: Anfernee Simons: 0 PTS
1626159: Justise Winslow: 0 PTS

1628978: Donte DiVincenzo: 21 PTS
203110: Draymond Green: 12 PTS
1626172: Kevon Looney: 14 PTS
202691: Klay Thompson: 23 PTS
1629673: Jordan Poole: 29 PTS
1630228: Jonathan Kuminga: 16 PTS
1629660: Ty Jerome: 2 PTS
203210: JaMychal Green: 2 PTS
1630237: Anthony Lamb: 4 PTS
1631116: Patrick Baldwin Jr.: 0 PTS
1630541: Moses Moody: 0 PTS
1631157: Ryan Rollins: 0 PTS
201939: Stephen Curry: 0 PTS
2738: Andre Iguodala: 0 PTS
1627780: Gary Payton II: 0 PTS
203952: Andrew Wiggins: 0

In [51]:
# Query nba.live.endpoints for the scoreboard of GAME_ID = '0022200931' (Blazers vs Warriors)
# Simple play by play loop to show the data usage
from nba_api.live.nba.endpoints import playbyplay
from nba_api.stats.static import players

pbp = playbyplay.PlayByPlay('0022200931')
line = "{action_number}: {period}:{clock} {player_id} ({action_type})"
actions = pbp.get_dict()['game']['actions'] #plays are referred to in the live data as 'actions'
for action in actions:
    player_name = ''
    player = players.find_player_by_id(action['personId'])
    if player is not None:
        player_name = player['full_name']
    print(line.format(action_number = action['actionNumber'], period = action['period'], clock = action['clock'], action_type = action['actionType'], player_id = player_name))

2: 1:PT12M00.00S  (period)
4: 1:PT11M58.00S Draymond Green (jumpball)
7: 1:PT11M46.00S Klay Thompson (2pt)
8: 1:PT11M46.00S Matisse Thybulle (block)
9: 1:PT11M45.00S  (rebound)
10: 1:PT11M34.00S Jordan Poole (2pt)
11: 1:PT11M31.00S  (rebound)
13: 1:PT11M16.00S Jerami Grant (3pt)
14: 1:PT11M13.00S Matisse Thybulle (rebound)
15: 1:PT11M11.00S Cam Reddish (turnover)
16: 1:PT11M11.00S Draymond Green (steal)
17: 1:PT10M54.00S Klay Thompson (2pt)
18: 1:PT10M52.00S Jerami Grant (rebound)
19: 1:PT10M45.00S Damian Lillard (2pt)
20: 1:PT10M26.00S Klay Thompson (foul)
22: 1:PT10M26.00S Klay Thompson (turnover)
23: 1:PT10M13.00S Matisse Thybulle (3pt)
25: 1:PT09M54.00S Kevon Looney (2pt)
27: 1:PT09M44.00S Matisse Thybulle (3pt)
29: 1:PT09M28.00S Kevon Looney (2pt)
30: 1:PT09M27.00S  (rebound)
32: 1:PT09M13.00S Jerami Grant (3pt)
33: 1:PT09M10.00S Draymond Green (rebound)
34: 1:PT08M56.00S Donte DiVincenzo (2pt)
35: 1:PT08M54.00S Kevon Looney (rebound)
36: 1:PT08M53.00S Kevon Looney (2pt)
37: 1:PT0